In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import os
from torch.utils.data import DataLoader, random_split


batch_size = 128


transform = transforms.Compose(
    [
    transforms.ToTensor(),
    #transforms.Normalize((0.1307,), (0.3081,))
    ])

trainset = torchvision.datasets.MNIST(os.getcwd(), train=True,download=True, transform=transform)
trainset, valset = random_split(trainset, [55000, 5000])
testset = torchvision.datasets.MNIST(os.getcwd(), train=False,download=True, transform=transform)

train_dataloader = DataLoader(trainset, batch_size=batch_size,shuffle=True)
val_dataloader = DataLoader(valset, batch_size=batch_size,shuffle=True)
test_dataloader = DataLoader(testset, batch_size=batch_size,shuffle=False)

  0%|          | 0/9912422 [00:00<?, ?it/s]

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.stack = nn.Sequential(
            nn.Conv2d(1,32,3),
            nn.ReLU(),
            nn.Conv2d(32,64,3),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout(0.25),
            nn.Flatten(),
            nn.Linear(64*12*12, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128,10),
            #nn.Softmax()
        )

    def forward(self, x):
        x = self.stack(x)
        return x

In [ ]:
#How to compute conv layers
layer = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3)
example = trainset[0][0]
layer(example).shape

torch.Size([32, 26, 26])

In [ ]:
import pytorch_lightning as pl
from torchmetrics import Accuracy

class MyLitModel(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.save_hyperparameters()
        self.model = Net()
        self.loss_fn = nn.CrossEntropyLoss()
        self.train_accuracy = Accuracy(task="multiclass", num_classes=10)
        self.val_accuracy = Accuracy(task="multiclass", num_classes=10)
        self.test_accuracy = Accuracy(task="multiclass", num_classes=10)
        
    
    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = self.loss_fn(logits, y)
        self.log("train_loss", loss, on_epoch=True, prog_bar=True)
        acc = self.train_accuracy(logits, y)
        self.log("train_accuracy", acc, on_epoch=True, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = self.loss_fn(logits, y)
        self.log("val_loss", loss)
        acc = self.val_accuracy(logits, y)
        self.log("val_accuracy", acc)

    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = self.loss_fn(logits, y)
        self.log("test_loss", loss)
        acc = self.test_accuracy(logits, y)
        self.log("test_accuracy", acc)

    def configure_optimizers(self):
        #optimizer = torch.optim.SGD(self.parameters(), lr=self.hparams.lr, momentum=0.9)
        optimizer = torch.optim.Adadelta(self.parameters())
        return optimizer

In [ ]:
from pytorch_lightning.callbacks import EarlyStopping

early_stopping_calback = EarlyStopping(monitor="train_loss", min_delta=0.1, mode="min")

In [ ]:
model = MyLitModel()

trainer = pl.Trainer(
    max_epochs=12,
    accelerator="auto",
    devices="auto",
    callbacks=early_stopping_calback
)

trainer.fit(model, train_dataloader, val_dataloader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: c:\Users\m.mikowski\Documents\light\tensor\4-MNIST\lightning_logs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type               | Params
------------------------------------------------------
0 | model          | Net                | 1.2 M 
1 | loss_fn        | CrossEntropyLoss   | 0     
2 | train_accuracy | MulticlassAccuracy | 0     
3 | val_accuracy   | MulticlassAccuracy | 0     
4 | test_accuracy  | MulticlassAccuracy | 0     
------------------------------------------------------
1.2 M     Trainable params
0         Non-trainable params
1.2 M     Total params
4.800     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

c:\Users\m.mikowski\Anaconda3\envs\env1\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:480: PossibleUserWarning: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.
  rank_zero_warn(
c:\Users\m.mikowski\Anaconda3\envs\env1\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:432: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
c:\Users\m.mikowski\Anaconda3\envs\env1\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:432: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [ ]:
trainer.test(model, test_dataloader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


c:\Users\m.mikowski\Anaconda3\envs\env1\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:432: PossibleUserWarning: The dataloader, test_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.9897000193595886     │
│         test_loss         │    0.03436831757426262    │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.03436831757426262, 'test_accuracy': 0.9897000193595886}]